# Assignment 4: RNN

In [6]:
from src import RNN 
from src import preprocessing 
import importlib
import numpy as np
importlib.reload(preprocessing)
importlib.reload(RNN)

<module 'src.RNN' from '/Users/annasanchezespunyes/Documents/GitHub/DD2424/lab4/src/RNN.py'>

### Read data

In [7]:
read_data = preprocessing.load_data("/Users/annasanchezespunyes/Documents/GitHub/DD2424/lab4/data/goblet_book.txt")

In [8]:
read_data.keys()

dict_keys(['data', 'char', 'K', 'char_to_ind', 'ind_to_char'])

### Compare gradient

In [129]:
sequence_length = 25
data = read_data['data']

input_chars = data[0:sequence_length]
target_chars = data[1: 1 + sequence_length]
X = RNN.encode(input_chars, read_data['char_to_ind'])
Y = RNN.encode(target_chars, read_data['char_to_ind'])

model = RNN.RNN(read_data['K'])
grads,_ = model.compute_gradients(X, Y)
grads['W'].shape

(5, 5)

In [128]:
h0 = np.zeros(shape=(5,1))
model_num = RNN.RNN(read_data['K'])
grad_num = model_num.ComputeGradsNum(X,Y,h0,1e-4)
grad_num['W'].shape

(5, 5)

In [117]:
model_num = RNN.RNN(read_data['K'])
grad_num = model_num.compute_grads_num(X,Y,1e-4)

100%|██████████| 5/5 [02:18<00:00, 27.76s/it]


In [132]:
def check_matrices(a_anal,a_num):
    assert a_anal.shape == a_num.shape
    matrix = np.abs(a_anal - a_num)
    print("Value maximum", matrix.max())
    return matrix


In [134]:
check_matrices(grad_num['W'],grads['W'])
print("HEllo")

Value maximum 0.0015772632395394553
HEllo


In [93]:
train_model = RNN.RNN(read_data['K'])
train_model.fit(read_data,sequence_length)

iter = 0 loss = 109.55066526768358
iter = 100 loss = 120.5727403487124
iter = 200 loss = 131.87182102017638
iter = 300 loss = 143.84329489233738
iter = 400 loss = 155.3850845495284
iter = 500 loss = 169.04278941139958
iter = 600 loss = 181.5907723774956
iter = 700 loss = 193.1438301989236
iter = 800 loss = 202.42266900264457
iter = 900 loss = 213.63340329727524
iter = 1000 loss = 223.1946236725861
iter = 1100 loss = 229.95004633368706
iter = 1200 loss = 238.73221641819242
iter = 1300 loss = 244.84684718576682
iter = 1400 loss = 251.11939376198393
iter = 1500 loss = 254.8054759202429
iter = 1600 loss = 257.8470633257868
iter = 1700 loss = 259.3560884912915
iter = 1800 loss = 262.97980491332544
iter = 1900 loss = 264.76538025833185
iter = 2000 loss = 264.00885322332016
iter = 2100 loss = 263.7622104479511
iter = 2200 loss = 264.49891180366234
iter = 2300 loss = 266.1826678494691
iter = 2400 loss = 266.63338179541205
iter = 2500 loss = 267.51230538496503
iter = 2600 loss = 271.46012600768

KeyboardInterrupt: 

In [16]:
def one_hot(character_index, number_distinct_characters):
    character_one_hot = np.zeros(shape=(number_distinct_characters,1))
    character_one_hot[character_index,0] = 1
    
    return character_one_hot

class RNN2(object):
    
    def __init__(self, k, m=5, sig=0.01, seed=42):
        self.k = k
        self.m = m
        self.ind_to_char = None
        self.char_to_ind = None
        self.param = {'U': np.random.normal(0, sig, size=(m, k)), 'V': np.random.normal(0, sig, size=(k, m)),
                      'W': np.random.normal(0, sig, size=(m, m)), 'b': np.zeros((m, 1)),
                      'c': np.zeros((k, 1))}
    
    def softmax(self, x):
        return np.exp(x) / np.sum(np.exp(x), axis=0)

    def forward(self, input_labels, h0):
        seq_length = input_labels.shape[1]
        p, h, a = [], [h0], []
        for t in range(seq_length):
            a.append(self.param['W']@h[-1]+self.param['U']@input_labels[:,[t]]+self.param['b'])
            h.append(np.tanh(a[t]))
            p.append(self.softmax(self.param['V']@h[-1]+self.param['c']))
        return p,h, a

    def calculate_loss(self,Y,p):
        loss = 0
        for t in range(len(p)):
            loss -= np.log(Y[:,[t]].T@p[t])[0,0]
        return loss
    
    def blank_parameters(self):
        return {'U': np.zeros(self.param['U'].shape), 'V': np.zeros(self.param['V'].shape),
                'W': np.zeros(self.param['W'].shape), 'b': np.zeros(self.param['b'].shape),
                'c': np.zeros(self.param['c'].shape)}
    
    def backward(self, X, Y, p, h, a):
        seq_length = X.shape[1]
        h0 = h[0]
        h = h[1:]
        grads = self.blank_parameters()
        grad_a = [None]*seq_length
        
         
        for t in range((seq_length-1),-1,-1):
            g = -(Y[:,[t]]-p[t]).T
            grads['V'] += g.T@h[t].T
            grads['c'] += g.T
            if t<(seq_length-1):
                dL_h = g@self.param['V']+grad_a[t+1]@self.param['W']
            else:
                dL_h = g@self.param['V']
            grad_a[t] = (dL_h@np.diag(1-np.square(h[t][:,0])))
            if t==0:
                grads['W'] += grad_a[t].T@h0.T
            else:
                grads['W'] += grad_a[t].T@h[t-1].T
            grads['U'] += grad_a[t].T@X[:,[t]].T
            grads['b'] += grad_a[t].T

        # Clipping gradients
        for parameter in ['b','c','U','W','V']:
            grads[parameter] = np.clip(grads[parameter], -5, 5)

        return grads   

In [17]:
def encode(input_text, char_to_ind):
    indices = [char_to_ind[char] for char in input_text]
    one_hot_encoding = (np.eye(len(char_to_ind.keys()))[indices]).T
    return one_hot_encoding

In [18]:
K = 80
m = 5
myRNN = RNN2(read_data['K'])
sequence_length = 25
data = read_data['data']

input_chars = data[0:sequence_length]
target_chars = data[1: 1 + sequence_length]
X = encode(input_chars, read_data['char_to_ind'])
Y = encode(target_chars, read_data['char_to_ind'])

In [19]:
h0 = np.zeros(shape=(m,1))
print("HW")
p, h, a = myRNN.forward(X,h0)
#loss = myRNN.calculate_loss(Y,p)
loss = myRNN.calculate_loss(Y,p)
print("Loss is", loss)
newGRADS = myRNN.backward(X, Y, p, h, a)

HW
Loss is 109.55059417856833


In [20]:
import copy
def ComputeGradsNum(RNN, X, Y, h0, h=1e-4):
   
    # Iterate parameters and compute gradients numerically
    
    GRADS = RNN.blank_parameters()
    for parameter in RNN.param.keys():
        for i in range(RNN.param[parameter].shape[0]):
            for j in range(RNN.param[parameter].shape[1]):
                RNN_try = copy.deepcopy(RNN)
                RNN_try.param[parameter][i,j] += h
                p, _, _ = RNN_try.forward(X, h0)
                loss2 = RNN_try.calculate_loss(Y,p)
                RNN_try.param[parameter][i,j] -= 2*h
                p, _, _ = RNN_try.forward(X, h0)
                loss1 = RNN_try.calculate_loss(Y,p)
                GRADS[parameter][i,j] = (loss2-loss1)/(2*h)
    
    return GRADS

In [21]:
newGRADS_num = ComputeGradsNum(myRNN, X, Y, h0)

In [22]:
for parameter in ['b','c','U','W','V']:
    error = abs(newGRADS_num[parameter]-newGRADS[parameter])
    mean_error = np.mean(error<1e-6)
    max_error = error.max()
    print('For '+parameter+', the % of absolute errors <1e-6 is '+str(mean_error*100)+ \
          ' and the maximum is '+str(max_error))

For b, the % of absolute errors <1e-6 is 100.0 and the maximum is 7.796818191074806e-10
For c, the % of absolute errors <1e-6 is 100.0 and the maximum is 8.039172660900817e-10
For U, the % of absolute errors <1e-6 is 100.0 and the maximum is 2.733765765844387e-10
For W, the % of absolute errors <1e-6 is 100.0 and the maximum is 2.42241289527545e-10
For V, the % of absolute errors <1e-6 is 100.0 and the maximum is 3.3293381755932633e-10
